In [20]:
"""
Memory
- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 항상 세 개의 이모티콘으로 답장하도록 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
"""

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage

# 1. LLM 초기화
chat = ChatOpenAI(temperature=0.1)

# 2. 예시 데이터 준비
examples = [
    {"movie": "탑건", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "emojis": "👨‍👨‍👦🔫🍝"},
    {"movie": "타이타닉", "emojis": "🚢💑🌊"},
    {"movie": "매트릭스", "emojis": "🕶️💊🤖"}
]

# 3. 영화 이모지 프롬프트
movie_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="당신은 영화 제목을 받아서 그 영화를 가장 잘 표현하는 3개의 이모지로 응답하는 AI입니다. 반드시 3개의 이모지만 사용해야 합니다."),
    FewShotChatMessagePromptTemplate(
        example_prompt=ChatPromptTemplate.from_messages([
            ("human", "영화: {movie}"),
            ("assistant", "{emojis}")
        ]),
        examples=examples
    ),
    ("human", "영화: {movie}")
])

# 4. 메모리 조회 프롬프트
memory_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""당신은 이전 대화 내용을 정확하게 분석하여 질문에 답변하는 AI입니다.

아래는 대화 내용입니다. 각 대화는 시간 순서대로 나열되어 있으며, 가장 위가 첫 번째 대화입니다:
{chat_history}

위 대화 내용을 바탕으로 다음 규칙에 따라 답변해주세요:
1. "첫 번째"는 가장 처음으로 물어본 영화를 의미합니다.
2. "마지막"은 가장 최근에 물어본 영화를 의미합니다.
3. "두 번째", "세 번째" 등은 순서대로 물어본 영화를 의미합니다.
4. 메모리 조회 질문과 그에 대한 답변은 순서에 포함하지 않습니다.
5. 동일한 영화가 여러 번 나오더라도 각각을 별개의 순서로 계산합니다.

질문에 답변할 때는 반드시 대화 기록을 처음부터 순서대로 확인하여 정확한 순서를 파악하세요."""),
    ("human", "{question}")
])

# 5. 메모리 초기화
memory = ConversationBufferMemory(
    return_messages=True
)

# 6. Chain 구성
movie_chain = movie_prompt | chat
memory_chain = memory_prompt | chat

def print_memory_state(title="현재 메모리 상태"):
    print("\n" + "="*50)
    print(f"📝 {title}")
    print("="*50)
    for idx, msg in enumerate(memory.chat_memory.messages, 1):
        role = "👤 사용자" if msg.type == "human" else "🤖 AI"
        print(f"{idx}. {role}: {msg.content}")
    print("="*50 + "\n")

# 7. Chain 실행 함수
def invoke_chain(input_text: str):
    print(f"\n🎬 입력: {input_text}")
    
    # 메모리 조회 질문인 경우
    if "물어본 영화" in input_text:
        # 대화 기록을 문자열로 변환
        chat_history = "\n".join([
            f"{'사용자' if msg.type == 'human' else 'AI'}: {msg.content}"
            for msg in memory.chat_memory.messages
        ])
        
        # 메모리 체인 실행
        result = memory_chain.invoke({
            "chat_history": chat_history,
            "question": input_text
        })
    # 영화 이모지 요청인 경우
    else:
        # 영화 체인 실행
        result = movie_chain.invoke({"movie": input_text})
        # 메모리에 대화 저장
        memory.chat_memory.add_user_message(f"영화: {input_text}")
        memory.chat_memory.add_ai_message(result.content)
    
    print(f"🤖 응답: {result.content}")
    print_memory_state()
    return result

# 8. 테스트 실행
result1 = invoke_chain("인셉션")
result2 = invoke_chain("어벤져스")
result3 = invoke_chain("명량")

# 9. 메모리 테스트
result4 = invoke_chain("첫 번째로 물어본 영화가 무엇인가요?")
result5 = invoke_chain("마지막으로 물어본 영화가 무엇인가요?")
result6 = invoke_chain("명량")
result7 = invoke_chain("세번째로 물어본 영화가 무엇인가요?")

# ---------------------------
# 주요 주석 요약
# - 두 개의 별도 체인 구성:
#   1. movie_chain: 영화 제목을 이모지로 변환
#   2. memory_chain: 대화 기록을 바탕으로 질문 답변
# - ConversationBufferMemory: 대화 기록 저장
# - 입력 텍스트에 따라 적절한 체인 선택
# ---------------------------


🎬 입력: 인셉션
🤖 응답: 🧠🌌🕰️

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️


🎬 입력: 어벤져스
🤖 응답: 🦸‍♂️🦸‍♀️🌌

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️
3. 👤 사용자: 영화: 어벤져스
4. 🤖 AI: 🦸‍♂️🦸‍♀️🌌


🎬 입력: 명량
🤖 응답: ⚓️🌊🚢

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️
3. 👤 사용자: 영화: 어벤져스
4. 🤖 AI: 🦸‍♂️🦸‍♀️🌌
5. 👤 사용자: 영화: 명량
6. 🤖 AI: ⚓️🌊🚢


🎬 입력: 첫 번째로 물어본 영화가 무엇인가요?
🤖 응답: 첫 번째로 물어본 영화는 '인셉션'입니다.

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️
3. 👤 사용자: 영화: 어벤져스
4. 🤖 AI: 🦸‍♂️🦸‍♀️🌌
5. 👤 사용자: 영화: 명량
6. 🤖 AI: ⚓️🌊🚢


🎬 입력: 마지막으로 물어본 영화가 무엇인가요?
🤖 응답: 마지막으로 물어본 영화는 '인셉션'입니다.

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️
3. 👤 사용자: 영화: 어벤져스
4. 🤖 AI: 🦸‍♂️🦸‍♀️🌌
5. 👤 사용자: 영화: 명량
6. 🤖 AI: ⚓️🌊🚢


🎬 입력: 명량
🤖 응답: ⚓️🌊🔥

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️
3. 👤 사용자: 영화: 어벤져스
4. 🤖 AI: 🦸‍♂️🦸‍♀️🌌
5. 👤 사용자: 영화: 명량
6. 🤖 AI: ⚓️🌊🚢
7. 👤 사용자: 영화: 명량
8. 🤖 AI: ⚓️🌊🔥


🎬 입력: 세번째로 물어본 영화가 무엇인가요?
🤖 응답: 세번째로 물어본 영화는 '인셉션'입니다.

📝 현재 메모리 상태
1. 👤 사용자: 영화: 인셉션
2. 🤖 AI: 🧠🌌🕰️
3. 👤 사용자: 영화: 어벤져스
4. 🤖 AI: 🦸‍♂️🦸‍♀️🌌
5. 👤 사용자: 영화: 명량
6. 🤖 AI: ⚓️🌊🚢
7

### Memory 챌린지 구현 이슈 정리

#### 1. MessagesPlaceholder 임포트 에러
- **문제**: `MessagesPlaceholder` 임포트 경로 오류
  ```python
  from langchain.schema import MessagesPlaceholder  # 오류
  ```
- **원인**: LangChain 업데이트로 인한 모듈 위치 변경
- **해결**: 올바른 임포트 경로 사용
  ```python
  from langchain.prompts import MessagesPlaceholder  # 정상
  ```

#### 2. 메모리 변수 타입 불일치
- **문제**: `variable history should be a list of base messages, got {'history': []}`
- **원인**: 
  - `RunnablePassthrough.assign`이 반환하는 메모리 데이터 구조가 예상과 다름
  - 딕셔너리 형태로 반환되는 메모리를 메시지 리스트로 처리하려고 시도
- **해결방법**:
  ```python
  # 수정 전: 메모리 로드 함수
  def load_memory(_):
      return memory.load_memory_variables({})
  
  # 수정 후: 올바른 형식으로 반환
  def load_memory(_):
      memory_data = memory.load_memory_variables({})
      return {"history": memory_data["history"]}
  ```

#### 3. 메모리 조회 응답 형식 문제
- **문제**: 메모리 조회 질문에도 이모지로 응답
- **원인**: 
  - 모든 입력을 동일한 이모지 생성 체인으로 처리
  - 질문 유형에 따른 분기 처리 부재
- **해결**:
  1. 별도의 프롬프트 템플릿 생성
     ```python
     # 영화 이모지용 프롬프트
     movie_prompt = ChatPromptTemplate.from_messages([...])
     
     # 메모리 조회용 프롬프트
     memory_prompt = ChatPromptTemplate.from_messages([...])
     ```
  2. 입력 텍스트에 따른 체인 분기 처리
     ```python
     def invoke_chain(input_text: str):
         if "물어본 영화" in input_text:
             # 메모리 조회 체인 실행
             return memory_chain.invoke({...})
         else:
             # 영화 이모지 체인 실행
             return movie_chain.invoke({...})
     ```

#### 4. 메모리 상태 확인 문제 ToDo : 조치 못함...
- **문제**: 대화가 진행되면서 메모리에 어떤 내용이 저장되는지 확인이 어려움
- **원인**: 
  - 메모리 상태를 시각적으로 확인할 수 있는 기능 부재
  - 디버깅과 문제 해결이 어려움
- **해결**:
  ```python
  def print_memory_state(title="현재 메모리 상태"):
      print("\n" + "="*50)
      print(f"📝 {title}")
      print("="*50)
      for idx, msg in enumerate(memory.chat_memory.messages, 1):
          role = "👤 사용자" if msg.type == "human" else "🤖 AI"
          print(f"{idx}. {role}: {msg.content}")
      print("="*50 + "\n")
  
  def invoke_chain(input_text: str):
      print(f"\n🎬 입력: {input_text}")
      # ... (체인 실행 로직) ...
      print(f"🤖 응답: {result.content}")
      print_memory_state()  # 매 상호작용 후 메모리 상태 출력
      return result
  ```
- **개선된 기능**:
  - 각 상호작용마다 입력과 응답을 명확히 표시
  - 현재까지의 전체 대화 기록 확인 가능
  - 이모지를 활용한 직관적인 역할 구분
  - 순차적인 대화 흐름 파악 용이

#### 5. 구현 시 주요 고려사항
1. **메모리 관리**
   - 대화 내용의 적절한 저장과 검색
   - 메시지 형식의 일관성 유지
   - 대화 맥락 보존

2. **프롬프트 설계**
   - 명확한 시스템 메시지
   - Few-shot 예제를 통한 응답 형식 지정
   - 질문 유형별 적절한 프롬프트 분리

3. **체인 구성**
   - 단순하고 명확한 체인 구조
   - 에러 처리와 타입 검사
   - 재사용 가능한 컴포넌트 설계

#### 6. 학습 내용
1. **LangChain 업데이트 대응**
   - 최신 버전의 모듈 구조 이해
   - 문서 참조의 중요성

2. **메모리 활용**
   - 대화 이력 관리 방법
   - 메모리 컴포넌트의 유연한 활용
   - 메모리 상태의 시각화와 디버깅
   - 대화 맥락의 실시간 모니터링

3. **LCEL 패턴**
   - 체인 구성의 모범 사례
   - 컴포넌트 간 데이터 흐름 관리

---
참조:
- https://python.langchain.com/docs/modules/memory/
- https://python.langchain.com/docs/expression_language/
- https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/ 